# Konkurs
Zadaniem konkursu jest tak zwany [skoring kredytowy](https://pl.wikipedia.org/wiki/Scoring_kredytowy) (ang. `credit_score`). Są dane historyczne klientów i na podstawie tych danych należy stwierdzić czy warto dawać kredyt klientowi czy nie.

Metryka sukcesu będzie: AUC.

## Kaggle
Konkurs będzie dostępny na Kaggle - [link do konkursu](https://www.kaggle.com/t/5007f8f6f6e548348b8633146ca651d7).

### Rejestracja
Jak jeszcze nie masz konta na Kaggle, to proszę [założ](https://www.kaggle.com/account/register). Pod czas rejestracji jest wymagane potwierdzenie SMS. Zawsze na moich poprzednich warsztatach było 1-2 osoby, które pod żadnym argumentem nie chcieli dawać swój telefon. Jeśli nie chcesz, to OK... w takim razie możesz wykonać zadanie tylko w noteeboki (jako zadanie domowe). Uczenstnicwo w konkursie jest opcjonalne. Z drugiej strony, Kaggle to jest największa społeczność uczenia maszynowego na świecie. Dlatego jeśli chcesz tym zająć się napoważniej, inspirować się od innych lub czas od czasu brać udział na Kaggle - to polecam założyć konto na Kaggle.

## Regułamin
1. Konkurs rozpoczyna się w poniedziałek, **13 listopada** i kończy się w niedziele, **19 listopada 23:59**.
2. Osoba która będzie na pierwszym miejscu dostaje nagrodę - Google Home.
3. Miejsca będą widoczne na [Leader Board](https://www.kaggle.com/c/dataworkshop-credit-score/leaderboard) przez cały czas.
4. Zbiór danych został podzielony na dwie części: publiczny i prywatny. Publiczy zbiór danych zawiera cech + odpowiedź, prywatny zbiór tyko cechę. Należy zrobić predykcję na prywatnym zbiorze i wysłać na Kaggle. Wtedy jak skończy się konkurs, ostateczny wynik będzie robiony na zbiorze prywatnym.
5. Każdy uczestnik może dodawać (`submit`) do 5 prób rozwiazań.
6. Osoba która wygrała opublikuje swoje rozwiązanie dla wszystkich i udostępni go podając link na slacku.
7. Nagroda będzie wysłana do 2 tygodni (i na pewno powinna przyjść przed końcem konkursu).

## Nagroda
![Google Home](../images/google_home.jpg)

In [1]:
import pandas as pd
import numpy as np
np.random.seed(2017)

from sklearn.metrics import auc, roc_curve, roc_auc_score

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import learning_curve, ShuffleSplit, KFold, train_test_split

import xgboost as xgb
import lightgbm as lgb

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', r"compiletime version 3.5 of module '_catboost' does not match runtime version 3.6")

    import catboost as ctb
    
from hyperopt import hp
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from functools import partial

import seaborn as sns

%matplotlib inline

## Dane

**Uwaga**. To są prawdziwe danne, dlatego część informacji jest nieznana (schowana) lub niejasna (jak magiczny kod itd). Dane można używać tylko w celach edukacyjnych!

- **customer_id** - ID klienta
- **b_id** - ID nieznane
- **b_request_date** - data czegość
- **date_confirmed** - data potwierdzenia
- **contract_type** - rodzaj umowy
- **contract_status** - status umowy
- **conract_open_date** - data podpisanie umowy
- **final_payment_date** - data ostatnie raty
- **real_date_closed_account** - faktyczna data zamknięcia konta
- **cred_limit** - limit kredytu
- **cur** - waluta
- **cur_debt** - bieżący dług
- **next_payment** - następna opłata
- **cur_balance** - balnas
- **duration_debt_days** - spóźnieni opłaty (w dniach)
- **cnt_delay_upto_5d** - ile razy spóźniony do 5 dni
- **cnt_delay_5d_29d** - ile razy spóźniony od 5 do 29 dni
- **cnt_delay_upto_30d** - ile razy spóźniony do 30 dni
- **cnt_delay_30d_59d** - ile razy spóźniony od 30 do 59 dni
- **cnt_delay_60d_89d** - ile razy spóźniony od 60 do 89 dni
- **cnt_delay_upto_90d** - ile razy spóźniony ponad 90 dni
- **str_start** - kod magicznej liczby
- **trustability_code** - zaszyfrowana wiarygodnosć klienta
- **cur_overdue_debt** - obecny dług
- **max_amount_debt** - maksymalny dług
- **interest_rate_loan** - stopa procentowa
- **code_frequency_payments** - kod płacenia
- **code_relationship_contract** - rodzaj relacji umowy
- **is_bad** - dobry czy zły klient :). To jest zmienna docelowa.

In [2]:
train = pd.read_hdf('../input/train.credit_score.h5')
test = pd.read_hdf('../input/test.credit_score.h5') #zbiór testowy bez odpowiedzi

## Analiza danych / Data analysis

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165454 entries, 0 to 193337
Data columns (total 29 columns):
customer_id                   165454 non-null int64
b_id                          165454 non-null int64
b_request_date                165454 non-null int64
date_confirmed                165454 non-null int64
contract_type                 165454 non-null int64
contract_status               165454 non-null int64
conract_open_date             165454 non-null int64
final_payment_date            154608 non-null float64
real_date_closed_account      80967 non-null object
cred_limit                    165454 non-null int64
cur                           165454 non-null int64
cur_debt                      133575 non-null object
next_payment                  75088 non-null object
cur_balance                   60343 non-null float64
duration_debt_days            134129 non-null float64
cnt_delay_upto_5d             165454 non-null int64
cnt_delay_5d_29d              165454 non-null int64

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4881 entries, 9 to 193335
Data columns (total 28 columns):
customer_id                   4881 non-null int64
b_id                          4881 non-null int64
b_request_date                4881 non-null int64
date_confirmed                4881 non-null int64
contract_type                 4881 non-null int64
contract_status               4881 non-null int64
conract_open_date             4881 non-null int64
final_payment_date            4672 non-null float64
real_date_closed_account      3278 non-null object
cred_limit                    4881 non-null int64
cur                           4881 non-null int64
cur_debt                      4102 non-null object
next_payment                  1936 non-null object
cur_balance                   1351 non-null float64
duration_debt_days            4042 non-null float64
cnt_delay_upto_5d             4881 non-null int64
cnt_delay_5d_29d              4881 non-null int64
cnt_delay_upto_30d            48

In [5]:
train.head()

,customer_id,b_id,b_request_date,date_confirmed,contract_type,contract_status,conract_open_date,final_payment_date,real_date_closed_account,cred_limit,...,cnt_delay_60d_89d,cnt_delay_upto_90d,str_start,trustability_code,cur_overdue_debt,max_amount_debt,interest_rate_loan,code_frequency_payments,code_relationship_contract,is_bad
0,14886,2,40575,40518,9,13,34004,36926.0,38798,5000,...,0,0,NaN,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,0,0,24,4,1,0.0
1,13016,2,40667,40588,6,13,35550,39202.0,39166,10000000,...,0,0,39120.0,X111111111111111111111111111111111111111111111...,0,0,24,3,1,0.0
2,498,2,40578,39809,99,52,36343,NaN,NaN,0,...,0,11,39743.0,XXXXXXXXXXXXXXXXXXXXXXXXXXXX5XXXXXXXXXXXXXX55X...,"1506,58","1506,58",0,7,1,0.0
3,1388,2,40592,39987,7,0,36344,40019.0,NaN,39831,...,0,0,40592.0,XXXXXXXXXXXXXXXXXXXX111XXXX1111X1111X1XX11XXXX...,0,0,0,7,1,0.0
4,25706,2,40564,39672,99,52,36345,NaN,NaN,0,...,0,3,40564.0,XXXXXXXXXXXXXXXXXXXXXXXXXXXXX5XXXXXXXXXXXX55XX...,"3151,55","3151,55",0,7,1,0.0


In [6]:
test.head()

,customer_id,b_id,b_request_date,date_confirmed,contract_type,contract_status,conract_open_date,final_payment_date,real_date_closed_account,cred_limit,...,cnt_delay_30d_59d,cnt_delay_60d_89d,cnt_delay_upto_90d,str_start,trustability_code,cur_overdue_debt,max_amount_debt,interest_rate_loan,code_frequency_payments,code_relationship_contract
9,5367,2,40600,40529,9,13,36816,38642.0,38203,5900,...,0,0,0,38203.0,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX,0,0,22,3,1
18,19171,1,40530,40512,7,0,37072,NaN,NaN,36364,...,0,0,0,40530.0,1111111111XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,0,"1969,72",0,3,1
31,8578,2,40608,40495,9,13,37229,39054.0,38894,16900,...,0,0,0,38894.0,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,0,0,22,3,1
42,13696,2,40586,40495,9,13,37348,39173.0,39092,30000,...,0,0,0,39051.0,X111111111111111111111111111111111111111111111...,0,0,22,3,1
43,29539,2,40607,40408,9,13,37350,39176.0,38772,10000,...,0,0,0,NaN,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX,0,0,22,3,1


In [7]:
train[train.is_bad == 1].describe()

,customer_id,b_id,b_request_date,date_confirmed,contract_type,contract_status,conract_open_date,final_payment_date,cred_limit,cur,...,cnt_delay_upto_5d,cnt_delay_5d_29d,cnt_delay_upto_30d,cnt_delay_30d_59d,cnt_delay_60d_89d,cnt_delay_upto_90d,str_start,interest_rate_loan,code_relationship_contract,is_bad
count,16683.000000,16683.000000,16683.000000,16683.000000,16683.000000,16683.000000,16683.000000,15379.000000,1.668300e+04,16683.000000,...,16683.000000,16683.000000,16683.000000,16683.000000,16683.000000,16683.000000,15660.000000,16683.000000,16683.000000,16683.0
mean,14024.374813,1.919079,40575.434994,40291.571840,18.341785,10.181023,39925.502667,41439.770531,5.250372e+04,0.001079,...,0.134268,0.568543,0.702811,0.208476,0.095486,0.481148,40279.886335,1.364083,1.008632,1.0
std,9451.507729,0.874747,55.127781,427.158117,28.196655,14.092381,608.809585,4994.395340,1.290183e+05,0.032831,...,0.693853,1.456609,1.757348,0.688984,0.412380,2.500076,450.730649,5.649190,0.191342,0.0
min,0.000000,1.000000,40427.000000,38478.000000,1.000000,0.000000,36821.000000,38138.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,38355.000000,0.000000,1.000000,1.0
25%,5941.000000,1.000000,40531.000000,40199.000000,9.000000,0.000000,39421.000000,39925.000000,8.000000e+03,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40153.000000,0.000000,1.000000,1.0
50%,12305.000000,2.000000,40574.000000,40486.000000,9.000000,13.000000,40172.000000,40579.000000,2.000000e+04,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40493.000000,0.000000,1.000000,1.0
75%,21485.000000,3.000000,40621.000000,40562.000000,9.000000,13.000000,40431.000000,41116.000000,5.000000e+04,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,40570.000000,0.000000,1.000000,1.0
max,35013.000000,3.000000,40725.000000,40720.000000,99.000000,61.000000,40675.000000,72958.000000,5.020326e+06,1.000000,...,18.000000,32.000000,32.000000,17.000000,16.000000,57.000000,40725.000000,84.000000,9.000000,1.0


In [8]:
train[train.is_bad == 0].describe()

,customer_id,b_id,b_request_date,date_confirmed,contract_type,contract_status,conract_open_date,final_payment_date,cred_limit,cur,...,cnt_delay_upto_5d,cnt_delay_5d_29d,cnt_delay_upto_30d,cnt_delay_30d_59d,cnt_delay_60d_89d,cnt_delay_upto_90d,str_start,interest_rate_loan,code_relationship_contract,is_bad
count,148771.000000,148771.000000,148771.000000,148771.000000,148771.000000,148771.000000,148771.000000,139229.000000,1.487710e+05,148771.000000,...,148771.000000,148771.000000,148771.000000,148771.000000,148771.000000,148771.000000,140389.000000,148771.000000,148771.000000,148771.0
mean,12886.609111,1.976057,40575.097129,40242.648016,16.903577,9.478642,39773.520774,41505.449533,7.537375e+04,0.004947,...,0.172184,0.554241,0.726418,0.200120,0.087846,0.517762,40226.470471,1.603807,1.015090,0.0
std,8958.774733,0.870134,55.198795,452.505712,26.420738,12.353073,615.780377,5702.735049,3.636312e+05,0.076135,...,0.872150,1.652232,2.081333,0.778837,0.427156,3.289122,481.312543,5.724866,0.275835,0.0
min,1.000000,1.000000,40431.000000,38356.000000,1.000000,0.000000,34004.000000,63.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,37935.000000,0.000000,1.000000,0.0
25%,5225.500000,1.000000,40529.000000,40066.000000,9.000000,0.000000,39280.000000,39834.000000,9.990000e+03,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40007.000000,0.000000,1.000000,0.0
50%,11472.000000,2.000000,40575.000000,40462.000000,9.000000,13.000000,39922.000000,40473.000000,2.298800e+04,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40471.000000,0.000000,1.000000,0.0
75%,19454.000000,3.000000,40621.000000,40552.000000,9.000000,13.000000,40322.000000,41052.000000,5.537000e+04,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40561.000000,0.000000,1.000000,0.0
max,35030.000000,3.000000,40722.000000,40711.000000,99.000000,61.000000,40702.000000,771529.000000,5.545522e+07,3.000000,...,26.000000,44.000000,58.000000,42.000000,27.000000,149.000000,40722.000000,228.000000,9.000000,0.0


In [9]:
def check_attribute(name):
    bad_rate = list(train[train.is_bad == 1][name].unique())
    good_rate = list(train[train.is_bad == 0][name].unique())
    if len(bad_rate) >= 100 or len(good_rate) >= 100:
        return
    only_in_good = [x for x in good_rate if x not in bad_rate]
    only_in_bad = [x for x in bad_rate if x not in good_rate]
    print('Attribute: ', name)
    print('Only good ', only_in_good)
    print('Only bad ', only_in_bad)
    
def check_unique(df):
    cols = list(df.columns.values)
    for col in cols:
        print('Column ', col, ' .Unique values count = ', df[col].unique().size)

In [10]:
check_unique(train)

Column  customer_id  .Unique values count =  30000
Column  b_id  .Unique values count =  3
Column  b_request_date  .Unique values count =  271
Column  date_confirmed  .Unique values count =  2143
Column  contract_type  .Unique values count =  11
Column  contract_status  .Unique values count =  7
Column  conract_open_date  .Unique values count =  2848
Column  final_payment_date  .Unique values count =  6791
Column  real_date_closed_account  .Unique values count =  4039
Column  cred_limit  .Unique values count =  29266
Column  cur  .Unique values count =  4
Column  cur_debt  .Unique values count =  42794
Column  next_payment  .Unique values count =  9646
Column  cur_balance  .Unique values count =  34838
Column  duration_debt_days  .Unique values count =  1237
Column  cnt_delay_upto_5d  .Unique values count =  25
Column  cnt_delay_5d_29d  .Unique values count =  38
Column  cnt_delay_upto_30d  .Unique values count =  44
Column  cnt_delay_30d_59d  .Unique values count =  27
Column  cnt_del

In [11]:
train_count = train[['customer_id', 'b_id']].groupby(['customer_id']).agg(['count'])
test_count = test[['customer_id', 'b_id']].groupby(['customer_id']).agg(['count'])

In [14]:
cols = list(train.select_dtypes(include=[np.number]).columns)
cols.remove('customer_id')
cols.remove('b_id')
cols.remove('b_request_date')
cols.remove('date_confirmed')
cols.remove('conract_open_date')
cols.remove('final_payment_date')
cols.remove('str_start')

for col in cols:
    check_attribute(col)

Attribute:  contract_type
Only good  [4, 14]
Only bad  []
Attribute:  contract_status
Only good  [21, 12]
Only bad  []
Attribute:  cur
Only good  [2, 3]
Only bad  []
Attribute:  cnt_delay_upto_5d
Only good  [26, 22, 17, 21, 13, 24, 15, 19, 20]
Only bad  []
Attribute:  cnt_delay_5d_29d
Only good  [24, 23, 29, 21, 28, 37, 41, 39, 30, 35, 31, 44]
Only bad  []
Attribute:  cnt_delay_upto_30d
Only good  [29, 33, 36, 40, 30, 24, 41, 37, 54, 31, 50, 35, 49, 34, 58]
Only bad  []
Attribute:  cnt_delay_30d_59d
Only good  [20, 14, 28, 24, 32, 16, 13, 42, 18, 36, 15, 31, 35]
Only bad  []
Attribute:  cnt_delay_60d_89d
Only good  [11, 15, 13, 14, 17, 19, 27, 12, 10, 20, 9]
Only bad  [16]
Attribute:  cnt_delay_upto_90d
Only good  [50, 34, 48, 66, 83, 60, 43, 67, 62, 55, 47, 85, 42, 56, 49, 65, 59, 100, 64, 63, 39, 61, 54, 58, 52, 44, 51, 91, 149, 107, 148, 137, 80, 87]
Only bad  []
Attribute:  interest_rate_loan
Only good  [45, 48, 7, 32, 51, 6, 59, 34, 28, 73, 44, 183, 228, 196, 211]
Only bad  []
Att

In [15]:
train.cur.unique()

array([0, 1, 2, 3])

In [16]:
train[train.is_bad == 0].interest_rate_loan.unique()

array([ 24,   0,  22,  19,  20,   9,  18,  11,  13,  16,  12,  17,   5,
        15,  23,  10,  25,  14,  21,  30,  29,  45,  26,  43,  48,  41,
        37,  36,   7,  40,  32,  38,   8,  27,  51,  50,  49,  31,   6,
        35,  57,  59,  34,  65,  77,  80,  84,  58,  52,  39,  28,  33,
        73,  55,  44, 183, 228,  47, 196, 211])

In [17]:
train[train.interest_rate_loan > 84.0].describe()

,customer_id,b_id,b_request_date,date_confirmed,contract_type,contract_status,conract_open_date,final_payment_date,cred_limit,cur,...,cnt_delay_upto_5d,cnt_delay_5d_29d,cnt_delay_upto_30d,cnt_delay_30d_59d,cnt_delay_60d_89d,cnt_delay_upto_90d,str_start,interest_rate_loan,code_relationship_contract,is_bad
count,6.000000,6.0,6.000000,6.000000,6.0,6.0,6.000000,6.000000,6.000000,6.0,...,6.0,6.0,6.0,6.0,6.0,6.0,2.000000,6.000000,6.0,6.0
mean,6913.000000,2.0,40579.000000,40544.833333,9.0,0.0,40537.333333,40565.333333,5500.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40590.500000,197.333333,1.0,0.0
std,6079.184024,0.0,77.529349,70.388683,0.0,0.0,72.714969,72.279089,2738.612788,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,129.400541,18.683326,0.0,0.0
min,890.000000,2.0,40493.000000,40461.000000,9.0,0.0,40461.000000,40491.000000,2000.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40499.000000,183.000000,1.0,0.0
25%,2818.750000,2.0,40512.750000,40484.500000,9.0,0.0,40471.250000,40499.750000,4250.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40544.750000,183.000000,1.0,0.0
50%,4962.500000,2.0,40576.500000,40553.000000,9.0,0.0,40539.500000,40565.500000,5000.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40590.500000,189.500000,1.0,0.0
75%,9980.250000,2.0,40635.000000,40599.000000,9.0,0.0,40594.250000,40623.750000,6500.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40636.250000,207.250000,1.0,0.0
max,16906.000000,2.0,40682.000000,40626.000000,9.0,0.0,40623.000000,40649.000000,10000.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40682.000000,228.000000,1.0,0.0


In [18]:
train.trustability_code.unique().size

44298

In [19]:
train[train.duration_debt_days.isnull()].head()

,customer_id,b_id,b_request_date,date_confirmed,contract_type,contract_status,conract_open_date,final_payment_date,real_date_closed_account,cred_limit,...,cnt_delay_60d_89d,cnt_delay_upto_90d,str_start,trustability_code,cur_overdue_debt,max_amount_debt,interest_rate_loan,code_frequency_payments,code_relationship_contract,is_bad
22,11332,3,40508,39405,7,13,37087,72822.0,39250,49000,...,0,0,39250.0,1111111111111111111111111111111111111111111111...,0,0,0,3,1,0.0
41,13065,3,40548,39441,7,13,37336,72858.0,38894,20000,...,0,0,38894.0,11XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX11XXXXXXXXXXXX...,0,0,0,3,1,0.0
45,4789,3,40548,39438,7,13,37361,72855.0,39076,3000,...,0,0,39076.0,1XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,0,0,0,3,1,0.0
50,27032,3,40614,40581,7,13,37434,72872.0,39250,7500,...,1,16,39250.0,555555555555555432A2A21121A1111111111111111111...,0,0,0,3,1,0.0
53,34116,3,40562,39460,7,13,37439,72876.0,39349,29450,...,1,19,39349.0,1XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX5555555...,0,0,0,3,1,0.0


## Przygotowanie danych / Data preparation

In [20]:
#typ wyjściowy / output type
train.is_bad = train.is_bad.astype(int)

In [21]:
import math
import datetime

EXCEL_DATE_SYSTEM_PC=1900

def fix_float(val):    
    if isinstance(val, float):
        return val
    else:
        return float(val.replace(',', '.'))
    
def stringify_float(val):
    if isinstance(val, str):
        return val
    if 'nan' == str(val).lower() or math.isnan(val):
        return 'unknown'
    if isinstance(val, float):
        return str(int(val))
    return str(val)

def to_date(val):
    date_string = fix_date(val)
    if date_string.find('/'):
        return datetime.datetime.strptime(date_string, '%m/%d/%Y')
    else:
        return datetime.datetime.strptime(date_string, '%Y-%m-%d')

def fix_date(val):
    if str(val).find('/') != -1 or str(val).find('-') != -1:
        return str(val)
    else:
        return excel_date(val).strftime('%m/%d/%Y')

def excel_date(val):
    return datetime.date(EXCEL_DATE_SYSTEM_PC, 1, 1) + datetime.timedelta(int(val)-2)

In [22]:
##floaty pisane , zamiast . / floats written using , instead of .
train.cur_debt = train.apply(lambda row: fix_float(row.cur_debt), axis=1)
train.cur_overdue_debt = train.apply(lambda row: fix_float(row.cur_overdue_debt), axis=1)
train.max_amount_debt = train.apply(lambda row: fix_float(row.max_amount_debt), axis=1)

test.cur_debt = test.apply(lambda row: fix_float(row.cur_debt), axis=1)
test.max_amount_debt = test.apply(lambda row: fix_float(row.max_amount_debt), axis=1)
test.cur_overdue_debt = test.apply(lambda row: fix_float(row.cur_overdue_debt), axis=1)

In [23]:
##object
train.code_frequency_payments = train.apply(lambda row: stringify_float(row.code_frequency_payments), axis=1)

test.code_frequency_payments = test.apply(lambda row: stringify_float(row.code_frequency_payments), axis=1)

In [24]:
##NaN
train.final_payment_date = train.final_payment_date.fillna(0)
train.real_date_closed_account = train.real_date_closed_account.fillna(0)
train.next_payment = train.apply(lambda row: 0 if row.cur_debt == 0.0 else row.next_payment, axis=1)
train.cur_balance = train.cur_balance.fillna(0)

test.final_payment_date = test.final_payment_date.fillna(0)
test.real_date_closed_account = test.real_date_closed_account.fillna(0)
test.next_payment = test.apply(lambda row: 0 if row.cur_debt == 0.0 else row.next_payment, axis=1)
test.cur_balance = test.cur_balance.fillna(0)

In [25]:
#daty / dates
train.final_payment_date = train.final_payment_date.astype(int)
test.final_payment_date = test.final_payment_date.astype(int)

date_cols = ['b_request_date', 'date_confirmed', 'conract_open_date', 'final_payment_date', 'real_date_closed_account']
for col in date_cols:
    train[col] = train.apply(lambda row: to_date(row[col]), axis=1)
    test[col] = test.apply(lambda row: to_date(row[col]), axis=1)
    
train.final_payment_date = pd.to_datetime(train.final_payment_date, errors='coerce')
test.final_payment_date = pd.to_datetime(test.final_payment_date, errors='coerce')

In [26]:
train[date_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165454 entries, 0 to 193337
Data columns (total 5 columns):
b_request_date              165454 non-null datetime64[ns]
date_confirmed              165454 non-null datetime64[ns]
conract_open_date           165454 non-null datetime64[ns]
final_payment_date          165453 non-null datetime64[ns]
real_date_closed_account    165454 non-null datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 12.6 MB


In [27]:
object_cols = list(train.select_dtypes(include=['object']).columns)

for column in object_cols:
    print(column)
    train[column+'_cat'], labels = pd.factorize(train[column])
    test[column+'_cat'], labels = pd.factorize(test[column])

next_payment
trustability_code
code_frequency_payments


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165454 entries, 0 to 193337
Data columns (total 32 columns):
customer_id                    165454 non-null int64
b_id                           165454 non-null int64
b_request_date                 165454 non-null datetime64[ns]
date_confirmed                 165454 non-null datetime64[ns]
contract_type                  165454 non-null int64
contract_status                165454 non-null int64
conract_open_date              165454 non-null datetime64[ns]
final_payment_date             165453 non-null datetime64[ns]
real_date_closed_account       165454 non-null datetime64[ns]
cred_limit                     165454 non-null int64
cur                            165454 non-null int64
cur_debt                       133575 non-null float64
next_payment                   120526 non-null object
cur_balance                    165454 non-null float64
duration_debt_days             134129 non-null float64
cnt_delay_upto_5d              165454 non-

In [29]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4881 entries, 9 to 193335
Data columns (total 31 columns):
customer_id                    4881 non-null int64
b_id                           4881 non-null int64
b_request_date                 4881 non-null datetime64[ns]
date_confirmed                 4881 non-null datetime64[ns]
contract_type                  4881 non-null int64
contract_status                4881 non-null int64
conract_open_date              4881 non-null datetime64[ns]
final_payment_date             4881 non-null datetime64[ns]
real_date_closed_account       4881 non-null datetime64[ns]
cred_limit                     4881 non-null int64
cur                            4881 non-null int64
cur_debt                       4102 non-null float64
next_payment                   3949 non-null object
cur_balance                    4881 non-null float64
duration_debt_days             4042 non-null float64
cnt_delay_upto_5d              4881 non-null int64
cnt_delay_5d_29d       

## Inżynieria cech / Feature engineering

In [32]:
#boolowskie / boolean
train.is_closed_account = train.apply(lambda row: row.real_date_closed_account != 0, axis=1)
train.is_overdue_debt_over_limit = train.apply(lambda row: row.cur_overdue_debt > row.cred_limit, axis=1)

test.is_closed_account = test.apply(lambda row: row.real_date_closed_account != 0, axis=1)
test.is_overdue_debt_over_limit = test.apply(lambda row: row.cur_overdue_debt > row.cred_limit, axis=1)

#bardzo ryzykowne / risky ones
#Attribute:  contract_type
#Only good  [4, 14]
#Only bad  []
#Attribute:  contract_status
#Only good  [21, 12]
#Only bad  []
#Attribute:  cur
#Only good  [2, 3]
#Only bad  []

train.is_good_cont_type = train.apply(lambda row: row.contract_type in [4, 14], axis=1)
train.is_good_cont_status = train.apply(lambda row: row.contract_status in [21, 12], axis=1)
train.is_good_cur = train.apply(lambda row: row.cur in [2, 3], axis=1)

test.is_good_cont_type = test.apply(lambda row: row.contract_type in [4, 14], axis=1)
test.is_good_cont_status = test.apply(lambda row: row.contract_status in [21, 12], axis=1)
test.is_good_cur = test.apply(lambda row: row.cur in [2, 3], axis=1)

#interest_rate_loan > 84.0
train.high_interest_rate_loan = train.apply(lambda row: row.interest_rate_loan > 84.0, axis=1)

test.high_interest_rate_loan = test.apply(lambda row: row.interest_rate_loan > 84.0, axis=1)

In [33]:
#intowe / int
train['delay_total'] = train.cnt_delay_upto_5d + train.cnt_delay_5d_29d + train.cnt_delay_upto_30d + train.cnt_delay_30d_59d + train.cnt_delay_60d_89d + train.cnt_delay_upto_90d
train['bal_total'] = train.cur_balance - train.cur_debt
train.bal_total = train.bal_total.fillna(0)

test['delay_total'] = test.cnt_delay_upto_5d + test.cnt_delay_5d_29d + test.cnt_delay_upto_30d + test.cnt_delay_30d_59d + test.cnt_delay_60d_89d + test.cnt_delay_upto_90d
test['bal_total'] = test.cur_balance - test.cur_debt
test.bal_total = test.bal_total.fillna(0)

#credit count
train['credit_count'] = train.apply(lambda row: train_count.loc[row.customer_id].iloc[0], axis=1)

test['credit_count'] = test.apply(lambda row: test_count.loc[row.customer_id].iloc[0], axis=1)

In [36]:
#boolowskie / bool
train.always_on_time = train.apply(lambda row: row.delay_total == 0, axis=1)

test.always_on_time = test.apply(lambda row: row.delay_total == 0, axis=1)

In [37]:
#ewentualne braki / missing values
train = train.fillna(0)
test = test.fillna(0)

## Model

In [38]:
def get_features():
    feats = list(train.columns.values)
    bad_feats = list(train.select_dtypes(include=['object', 'datetime']).columns.values) + ['is_bad', 'customer_id']
    return [f for f in feats if f not in bad_feats]

def get_X(df):
    return df[get_features()]

def get_y(df):
    return df['is_bad'];    

In [39]:
X = get_X(train)
y = get_y(train)

from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.3, random_state=2017)

In [ ]:
def objective(space):
    
    linreg_params = {
        'tol': space['tol'],
        'C': space['C'],
        'random_state': int(space['seed']),
        'max_iter': 550,
        'class_weight': 'balanced',
        'solver': space['solver'],
        'n_jobs': -1
    }
    
    score = 0
    model = LogisticRegression(**linreg_params)
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        score = roc_auc_score(y_test, y_pred)
        print("CURRENT SPLIT SCORE: {0}".format(score))
    
    print("SCORE: {0}".format(score))    
    return{'loss':score, 'status': STATUS_OK }
    
space ={
    'tol': hp.uniform('x_tol', 0.00001, 0.001),
    'C': hp.uniform ('x_C', 0.1, 1.0),
    'seed': hp.quniform ('x_seed', 0, 10000, 50),
    'solver': hp.choice('x_solver ', ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'])
}

trials = Trials()
best_params_linreg = fmin(fn=objective,
            space=space,
            algo=partial(tpe.suggest, n_startup_jobs=1),
            max_evals=20,
            trials=trials)

print("The best params: ", best_params_linreg)

/home/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


CURRENT SPLIT SCORE: 0.538431753315693
CURRENT SPLIT SCORE: 0.538711617742358
CURRENT SPLIT SCORE: 0.540178140554552
SCORE: 0.540178140554552
CURRENT SPLIT SCORE: 0.5384971208214621
CURRENT SPLIT SCORE: 0.538807820159765
CURRENT SPLIT SCORE: 0.5405516378226541
SCORE: 0.5405516378226541


/home/jupyter/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/jupyter/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


CURRENT SPLIT SCORE: 0.5617752228950472


/home/jupyter/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/jupyter/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


...

CURRENT SPLIT SCORE: 0.561752817446042


/home/jupyter/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/jupyter/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


CURRENT SPLIT SCORE: 0.5599900726625946


/home/jupyter/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/jupyter/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


CURRENT SPLIT SCORE: 0.5621866214087042
SCORE: 0.5621866214087042
CURRENT SPLIT SCORE: 0.5423912191948579
CURRENT SPLIT SCORE: 0.5436465824012875
CURRENT SPLIT SCORE: 0.547616467597647
SCORE: 0.547616467597647
CURRENT SPLIT SCORE: 0.5385540118601599
CURRENT SPLIT SCORE: 0.5396720124292269


## Submission
Trzeba przygotować plik który będzie zawierał: `customer_id` i `is_bad`.

In [ ]:
from hyperopt import space_eval
best_params_linreg = space_eval(space, best_params_linreg)

In [ ]:
linreg_params = {
    'tol': best_params_linreg['tol'],
    'C': best_params_linreg['C'],
    'random_state': int(best_params_linreg['seed']),
    'max_iter': 550,
    'class_weight': 'balanced',
    'solver': best_params_linreg['solver'],
    'n_jobs': -1    
}

In [ ]:
def predict(model):
    model.fit(X, y)
    X_test = get_X(test)
    return model.predict(X_test)

In [ ]:
model = LogisticRegression(**linreg_params)
y_pred = predict(model)

In [ ]:
import time
ts = str(int(time.time()))
test['is_bad'] = y_pred
test[ ['customer_id', 'is_bad'] ].to_csv('../output/model_dummy_' + ts + '.csv', index=False) #0.49409